In [11]:
import requests
import pandas as pd

In [19]:
API_KEY = "3"  # Clé publique par défaut
LEAGUE_ID = "4334"
SEASON = "2025-2026"

url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/lookuptable.php?l={LEAGUE_ID}&s={SEASON}"

print(f"🔍 Requête vers : {url}")

try:
    response = requests.get(url)
    response.raise_for_status()  # Lève une exception si erreur HTTP

    # Vérifie si le contenu est vide
    if not response.text.strip():
        print("⚠️ Réponse vide de l'API.")
    else:
        try:
            data = response.json()
            if "table" in data and data["table"]:
                print("✅ Données récupérées avec succès :\n")
                for team in data["table"]:
                    print(f"{team['intRank']}. {team['strTeam']} - {team['intPoints']} pts")
            else:
                print("⚠️ Pas de données de classement trouvées pour cette saison.")
        except ValueError:
            print("⚠️ Réponse non JSON. Voici le contenu brut :")
            print(response.text[:500])  # affiche les 500 premiers caractères
except requests.RequestException as e:
    print(f"🚨 Erreur de requête : {e}")

🔍 Requête vers : https://www.thesportsdb.com/api/v1/json/3/lookuptable.php?l=4334&s=2025-2026
✅ Données récupérées avec succès :

1. Paris SG - 20 pts
2. Marseille - 18 pts
3. Monaco - 17 pts
4. Strasbourg - 16 pts
5. Lens - 16 pts


In [21]:
def get_team_rank(team_name):
    """Récupère le rang d'une équipe dans le classement de TheSportsDB."""
    url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/lookuptable.php?l={LEAGUE_ID}&s={SEASON}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    if "table" not in data or not data["table"]:
        raise ValueError("Aucune donnée de classement trouvée.")

    for team in data["table"]:
        if team_name.lower() in team["strTeam"].lower():
            return int(team["intRank"])
    raise ValueError(f"Équipe '{team_name}' non trouvée dans la ligue.")

def proba_victoire(rang_A, rang_B):
    """Calcule les probabilités de victoire basées sur le classement."""
    pA = rang_B / (rang_A + rang_B)
    pB = rang_A / (rang_A + rang_B)
    return round(pA, 2), round(pB, 2)

def predire_match(equipe_A, equipe_B):
    """Prédit le vainqueur d’un match selon les rangs du classement."""
    rang_A = get_team_rank(equipe_A)
    rang_B = get_team_rank(equipe_B)

    pA, pB = proba_victoire(rang_A, rang_B)
    prediction = equipe_A if pA > pB else equipe_B

    print(f"Match : {equipe_A} (#{rang_A}) vs {equipe_B} (#{rang_B})")
    print(f"Probabilité {equipe_A} : {pA*100}% | {equipe_B} : {pB*100}%")
    print(f"➡️ Pronostic : victoire de {prediction}")

# Exemple d'utilisation :
predire_match("Paris SG", "Marseille")

Match : Paris SG (#1) vs Marseille (#2)
Probabilité Paris SG : 67.0% | Marseille : 33.0%
➡️ Pronostic : victoire de Paris SG
